In [1]:
import os 
import time
import numpy as np
import tensorflow as tf
from tensorflow.python.ops import summary_ops_v2
from tensorflow import keras
from tensorflow.keras import datasets,layers,models,optimizers,metrics

C:\Users\jianghaitao1\AppData\Local\Continuum\anaconda3\envs\tf2\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\jianghaitao1\AppData\Local\Continuum\anaconda3\envs\tf2\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\jianghaitao1\AppData\Local\Continuum\anaconda3\envs\tf2\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", n

In [2]:
# 0:默认值，打印所有信息 1：屏蔽INFO信息 2：屏蔽INFO信息和WARNING信息 3：屏蔽INFO,WARNING,ERROR信息
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2' 

In [3]:
def mnist_datasets():
    (x_train,y_train),(x_test,y_test) = datasets.mnist.load_data()
    #numpy默认是float64,tf默认是float32
    x_train,x_test = x_train/np.float32(255),x_test/np.float32(255)
    y_train,y_test = y_train.astype(np.int64),y_test.astype(np.int64)
    train_dataset = tf.data.Dataset.from_tensor_slices((x_train,y_train))
    test_dataset = tf.data.Dataset.from_tensor_slices((x_test,y_test))
    return train_dataset,test_dataset

In [4]:
train_ds,test_ds = mnist_datasets()

In [5]:
train_ds = train_ds.shuffle(60000).batch(100)
test_ds = test_ds.batch(100)

In [6]:
# 构建模型
model = tf.keras.Sequential([
    layers.Reshape(target_shape=[28,28,1],input_shape=[28,28]),
    layers.Conv2D(2,5,padding='same',activation=tf.nn.relu),
    layers.MaxPooling2D((2,2),(2,2),padding='same'),
    layers.Conv2D(4,5,padding='same',activation=tf.nn.relu),
    layers.MaxPool2D((2,2),(2,2),padding='same'),
    layers.Flatten(),
    layers.Dense(32,activation=tf.nn.relu),
    layers.Dropout(rate=0.4),
    layers.Dense(10)
])

In [7]:
optimizer = optimizers.SGD(lr=0.01,momentum=0.5)

In [8]:
compute_loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

In [9]:
compute_accuracy = tf.keras.metrics.SparseCategoricalAccuracy()

In [10]:
def train_one_step(model,optimizer,images,labels):
    with tf.GradientTape() as tape:
        logits = model(images,training=True)
        loss = compute_loss(labels,logits)
        compute_accuracy(labels,logits)
    grads = tape.gradient(loss,model.trainable_variables)
    optimizer.apply_gradients(zip(grads,model.trainable_variables))
    return loss

In [11]:
def train(model,optimizer,dataset,log_freq=50):
    
    avg_loss = metrics.Mean('loss',dtype=tf.float32)
    
    for images,labels in dataset:
        loss = train_one_step(model,optimizer,images,labels)
        avg_loss(loss)
        if tf.equal(optimizer.iterations % log_freq,0):
            train_summary_writer = tf.summary.create_file_writer('../../tf_ckpts/tf_model1/logs')
            with train_summary_writer.as_default():
#                 summary_ops_v2.scalar('loss',avg_loss.result(),step=optimizer.iterations)
#                 summary_ops_v2.scalar('accuracy',compute_accuracy.result(),step=optimizer.iterations)
                tf.summary.scalar('loss',avg_loss.result(),step=optimizer.iterations)
                tf.summary.scalar('accuracy',compute_accuracy.result(),step=optimizer.iterations)

            print('step:',int(optimizer.iterations),'loss:',avg_loss.result().numpy(),'acc:',compute_accuracy.result().numpy())
            avg_loss.reset_states()
            compute_accuracy.reset_states()

In [12]:
def test(model,dataset,step_num):
    avg_loss = metrics.Mean('loss',dtype=tf.float32)
    for (images,labels) in dataset:
        logits = model(images,training=False)
        avg_loss(compute_loss(labels,logits))
        compute_accuracy(labels,logits)
        
#     summary_ops_v2.scalar('loss',avg_loss.result(),step=step_num)
#     summary_ops_v2.scalar('accuracy',compute_loss.result(),step=step_num)
    
    print('Model test set loss:{:0.4f} accuracy:{0.2f}'.format(avg_loss.result(),compute_accuracy.result()*100))
    print('loss:',avg_loss.result(),'acc:',compute_accuracy.result())

In [13]:
MODEL_DIR = '../../tf_ckpts/tf_model1'

In [14]:
def apply_clean():
    if tf.io.gfile.exists(MODEL_DIR):
        print('Removing existing model dir:{}'.format(MODEL_DIR))
        tf.io.gfile.rmtree(MODEL_DIR)
apply_clean()
checkpoint_dir = os.path.join(MODEL_DIR,'checkpoints')
checkpoint_prefix = os.path.join(checkpoint_dir,'ckpt')
checkpoint = tf.train.Checkpoint(model=model,optimizer=optimizer)
checkpoint.restore(tf.train.latest_checkpoint(checkpoint_dir))

In [15]:
NUM_TRAIN_EPOCHS = 5
for i in range(NUM_TRAIN_EPOCHS):
    start = time.time()
    train(model, optimizer, train_ds, log_freq=500)
    end = time.time()
    print('Train time for epoch #{} ({} total steps): {}'.format(
        i + 1, int(optimizer.iterations), end - start))
    checkpoint.save(checkpoint_prefix)
    print('saved checkpoint')
    
export_path = os.path.join(MODEL_DIR,'export')
tf.saved_model.save(model,export_path)
print('saved SavedModel for exporting')

step: 500 loss: 1.3838977 acc: 0.52672
Train time for epoch #1 (600 total steps): 13.758132457733154
saved checkpoint
step: 1000 loss: 0.6197652 acc: 0.79432
Train time for epoch #2 (1200 total steps): 14.218112468719482
saved checkpoint
step: 1500 loss: 0.49627706 acc: 0.83322
Train time for epoch #3 (1800 total steps): 15.416229486465454
saved checkpoint
step: 2000 loss: 0.4229736 acc: 0.85702
Train time for epoch #4 (2400 total steps): 15.83239197731018
saved checkpoint
step: 2500 loss: 0.38794577 acc: 0.87186
step: 3000 loss: 0.36224905 acc: 0.8869
Train time for epoch #5 (3000 total steps): 15.838350296020508
saved checkpoint
saved SavedModel for exporting
